# Grids in BEM++

This tutorial demonstrates basic features of dealing with grids in BEM++. Simple grids can be easily created using built-in commands. More complicated grids can be imported in the Gmsh format.

## Creation of basic grid objects

Let us create our first grid, a simple regular sphere.

In [5]:
import bempp.api
grid = bempp.api.shapes.regular_sphere(3)

The command `regular_sphere` creates a sphere by refining a base octahedron. The number of elements in the sphere is given by `nelements = 8 * 4**n`, where `n` is the refinement level. We can plot the grid with the following command.

In [6]:
grid.plot()

This uses `Gmsh` to plot the grid externally. In order to work please make sure that `Gmsh` is installed and the command `gmsh` is available in the path. The following picture shows the sphere. <img src="https://raw.githubusercontent.com/bempp/bempp/development/examples/basic_concepts/sphere.png">

Another way to create a sphere is by specifying the width of the elements. The command

    grid = bempp.api.shapes.sphere(h=0.1)
    
will create an unstructured spherical grid with a grid size of roughly 0.1. Note that in order for this command to succeed `Gmsh` as grid generator must be installed. The `shapes` module contains functions for spheres, ellipsoids, cubes and the Nasa almond shape.

Sometimes, it is desired to create a regular structured 2d grid (such as a screen). For this BEM++ offers the functino `bempp.api.structured_grid`. The help text of this function gives more detail on its use.

## Creating grids from connectivity data

Quite often a grid is given in the form of connectivity data, that is an array containing the nodes and another array containing the element defintions from the nodes. Consider the following definition of vertices.

    vertices = np.array([[0,1,1,0],
                         [0,0,1,1],
                         [0,0,0,0]])
                        
The array `vertices` contains the (x,y,z) coordinates of the four vertices of the unit square in the x-y plane.

We now define two elements by specifiying how the vertices are connected.

    elements = np.array([[0,1],
                         [1,2],
                         [3,3]])

The first element connects the vertices 0, 1 and 3. The second element connects the vertices 1, 2 and 3. To create a grid from these two elements we simply call the following command.

    grid = bempp.api.grid_from_element_data(vertices,elements)
    
**Please note that BEM++ assumes that each element is defined such that the normal direction obtained with the right-hand rule is outward pointing.** Elements with inward pointing normals can easily be a source for errrors in computations. Normal directions can be visually checked for example by loading a mesh in `Gmsh` and displaying the normals.

Also, **it is not guaranteed that elements are stored in the grid object using the same numbering as during insertion.** Further down we will explain this in detail. To find out the insertion index of a vertex or an element the methods `grid.vertex_insertion_index` and `grid.element_insertion_index` are provided.

## Importing mesh data

In [9]:
from bempp.grid import grid_from_sphere
from bempp import function_space

grid = grid_from_sphere(2)
pw_constant = function_space(grid, "DP", 0)
pw_linear = function_space(grid, "P", 1)

Alternatively, a grid can be created from data using ``bempp.grid.grid_from_element_data`` or from a ``.msh`` file using ``bempp.import_grid``. For more information, run:

    from bempp.grid import grid_from_element_data
    help(grid_from_element_data)
    
or:

    from bempp import import_grid
    help(import_grid)
    

## Using the Grid

First, we'll print the number of entities of the grid. ``entity_count`` takes the codimension as its input.

In [3]:
print("Number of zero dimensional entities (vertices): %s" % grid.leaf_view.entity_count(2))
print("Number of one dimensional entities (edges): %s" % grid.leaf_view.entity_count(1))
print("Number of two dimensional entities (triangles): %s" % grid.leaf_view.entity_count(0))

Number of zero dimensional entities (vertices): 66
Number of one dimensional entities (edges): 192
Number of two dimensional entities (triangles): 128


If ``gmsh`` is installed, we can plot the grid. This will open in a seperate window.

In [4]:
grid.plot()

We can iterate over the grid using the ``entity_iterator``. Let's print the co-ordinates of each grid point with x-coordinate 0 and a positive y-coordinate.

In [5]:
for vertex in grid.leaf_view.entity_iterator(2):
    corners = vertex.geometry.corners
    if corners[0,0] == 0. and corners[1,0]>0:
        print("( %s , %s , %s )"% (corners[0,0], corners[1,0], corners[2,0]))

( 0.0 , 1.0 , 0.0 )
( 0.0 , 0.707106781187 , 0.707106781187 )
( 0.0 , 0.707106781187 , -0.707106781187 )
( 0.0 , 0.923879532511 , 0.382683432365 )
( 0.0 , 0.382683432365 , 0.923879532511 )
( 0.0 , 0.382683432365 , -0.923879532511 )
( 0.0 , 0.923879532511 , -0.382683432365 )


# Using the Function Spaces

First let's print the number of degrees of freedom (dofs) for each function space.

For the ``pw_constant`` space, the dofs will correspond to triangles.

For the ``pw_linear`` space, the dofs will correspond to vertices.

In [6]:
print("pw_constant dofs: %s" % pw_constant.global_dof_count)
print("pw_linear dofs: %s" % pw_linear.global_dof_count)

pw_constant dofs: 128
pw_linear dofs: 66


The function spaces can be used to define operators. Each operator takes the domain, range and dual_to_range spaces as its inputs (in that order).

In [7]:
from bempp.operators.boundary.sparse import identity

id_op = identity(pw_constant, pw_constant, pw_constant)

We can create functions in each function space. First, let's create one from the function
$$
f(x,n) = x_1
$$
and, if we have ``gmsh`` installed, plot it.

In [8]:
from bempp import GridFunction

def my_func(x, n, domain_index, result):
    result[0] = x[0]
    
my_grid_function = GridFunction(pw_constant, dual_space = pw_constant, fun = my_func, compex_data = True)
my_grid_function.plot()

Alternatively, we can create this function in the ``pw_linear`` function space.

In [7]:
my_grid_function = GridFunction(pw_linear, dual_space = pw_linear, fun = my_func, compex_data = True)
my_grid_function.plot()

Next, we'll make a grid function from some data. In this example, we'll use some random numbers.

In [10]:
import random
fake_data = np.array([random.random() for i in range(pw_constant.global_dof_count)])

my_grid_function = GridFunction(pw_constant, coefficients = fake_data)
my_grid_function.plot()

The data from a grid function can be exported to a file. The same command can be used to export a grid.

In [18]:
from bempp import export

export(grid_function = my_grid_function, file_name = "random_data.msh")
export(grid=grid, file_name = "sphere_grid.msh")